In [5]:
from nuscenes.nuscenes import NuScenes

import numpy as np
import cv2
import os

from typing import Tuple
from pprint import pprint

from pointcloud_tools import get_calibrated_pointcloud, compute_length_and_degree



nusc = NuScenes(version='v1.0-mini', dataroot='data/v1.0-mini', verbose=False)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)

In [7]:
def get_rings(ego_center: float = 6.,
ring_width: float = 1.,
last_ring: float = 100,
growth_rate: float = 1.1
) -> np.ndarray:
    ring_size = ego_center
    rings = [ring_size]

    while ring_size < last_ring:
        ring_size += ring_width
        ring_width *= growth_rate
        rings.append(ring_size)
    return np.array(rings)


In [8]:
angle_resolution = 3
ring_width, growth_rate = .25, 1.025
rings = get_rings(ring_width=ring_width, growth_rate=growth_rate)
angles = np.arange(0, 360, angle_resolution)
rings.shape, angles.shape

((96,), (120,))

In [17]:
def count_points_per_cell(pcd: np.ndarray, angles: np.ndarray, circles: np.ndarray) -> np.ndarray:
        vector_lengths, pcd_angles = compute_length_and_degree(pcd)

        angle_correction = np.select([pcd_angles < 0], [360], default=0)
        pcd_angles += angle_correction
        
        angle_bins = np.digitize(pcd_angles, angles)
        radius_bins = np.digitize(vector_lengths, circles)

        occupancy_cells = np.zeros((circles.shape[0]+ 1, angles.shape[0] + 1))
        for radius_bin, angle_bin in zip(radius_bins, angle_bins):
            occupancy_cells[radius_bin, angle_bin] += 1
        return occupancy_cells

def filter_pointcloud(pcd: np.ndarray, min_height: float = 0., min_x: float = 1., min_y: float = 1.) -> np.ndarray:
    pcd = pcd[np.where(pcd[:, 2] > min_height)]
    pcd = pcd[np.where(pcd[:, 2] < 3.)]
    pcd = pcd[np.where(pcd[:, 1] > min_x)]
    pcd = pcd[np.where(pcd[:, 0] > min_y)]
    return pcd

In [18]:
radar_pcds = []
radar_list = ["RADAR_FRONT", "RADAR_FRONT_LEFT", "RADAR_BACK_LEFT", "RADAR_BACK_RIGHT", "RADAR_FRONT_RIGHT"]
for sensor in radar_list:
    _, pcd_ego = get_calibrated_pointcloud(nusc, my_sample, sensor_name=sensor)
    pcd_ego = filter_pointcloud(pcd_ego)
    radar_pcds.append(pcd_ego)
_, lidar_pcd_ego = get_calibrated_pointcloud(nusc, my_sample, sensor_name="LIDAR_TOP")
lidar_pcd_ego = filter_pointcloud(lidar_pcd_ego)


In [19]:
points_per_cell = count_points_per_cell(pcd=lidar_pcd_ego, angles=angles, circles=rings)
occupied = np.select([points_per_cell > 0], [1.], default=0.)

cv2.namedWindow("Occupancy Lidar", cv2.WINDOW_NORMAL) 
cv2.resizeWindow("Occupancy Lidar", 1280, 1280) 
cv2.imshow("Occupancy Lidar", occupied)

laplacian = cv2.Laplacian(occupied,cv2.CV_64F)

edges = cv2.Canny(occupied.astype(np.uint8),0,255)

cv2.namedWindow("Laplacian Lidar", cv2.WINDOW_NORMAL) 
cv2.resizeWindow("Laplacian Lidar", 1280, 1280) 
cv2.imshow("Laplacian Lidar", laplacian)

cam_front = nusc.get('sample_data', my_sample['data']['CAM_FRONT'])
file_path = os.path.join("data/v1.0-mini/", cam_front["filename"])
image_front = cv2.imread(file_path)
cv2.namedWindow("Image Front", cv2.WINDOW_NORMAL) 
cv2.resizeWindow("Image Front", 1280, 1280) 
cv2.imshow("Image Front", image_front)

cam_back = nusc.get('sample_data', my_sample['data']['CAM_BACK'])
file_path = os.path.join("data/v1.0-mini/", cam_back["filename"])
image_back = cv2.imread(file_path)
cv2.namedWindow("Image Back", cv2.WINDOW_NORMAL) 
cv2.resizeWindow("Image Back", 1280, 1280) 
cv2.imshow("Image Back", image_back)


cv2.waitKey(0)
cv2.destroyAllWindows()


In [13]:
for pcd in radar_pcds:
    points_per_cell = count_points_per_cell(pcd=pcd, angles=angles, circles=rings)
    occupied = np.select([points_per_cell > 0], [1.], default=0.)
    cv2.namedWindow("Occupancy Lidar", cv2.WINDOW_NORMAL) 
    cv2.resizeWindow("Occupancy Lidar", 1280, 1280) 
    cv2.imshow("Occupancy Lidar", occupied)
    laplacian = cv2.Laplacian(occupied,cv2.CV_64F)

    cv2.namedWindow("Laplacian Lidar", cv2.WINDOW_NORMAL) 
    cv2.resizeWindow("Laplacian Lidar", 1280, 1280) 
    cv2.imshow("Laplacian Lidar", laplacian)

    cv2.waitKey(0)
    cv2.destroyAllWindows()